<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/Konlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install konlpy
!pip install sklearn

In [0]:
import pandas as pd
import pickle
import konlpy
import re
import sklearn as sk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize


In [0]:
with open('/content/drive/My Drive/comment_analysis/data/review.pickle', 'rb') as f:
    review = pickle.load(f)

In [0]:
review.columns = ['ranking','date','desc','asc']
review_tidy = review.applymap(lambda x:re.sub('[\[\]!"#$%&\'()*+,/:;<=>?@\^_`{|}~-✔️🙏🏻👍🤗💕😘]|BEST\n|재구매\n','',x))

In [0]:
review_ranking = list(review_tidy['ranking'])
review_rank_sents = sum([sent.split('\n') for sent in review_ranking],[])

In [0]:
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from konlpy.tag import Okt

In [0]:
twitter = Okt()
review_rank_noun = [twitter.nouns(sent) for sent in review_rank_sents]

In [0]:
corpus = [" ".join(noun) for noun in review_rank_noun] 
corpus = list(filter(lambda x: len(x)!=0,corpus))

출처: https://excelsior-cjh.tistory.com/93 [EXCELSIOR]

In [0]:
# 문장 간 연결 여부를 tf-idf * tf-idf matrix로 판단
# 각 문장 간 중요도를 1로 맞추기 위하여 normalize
vectorizer = TfidfVectorizer()
tfidf_mat = vectorizer.fit_transform(corpus).toarray()
graph_tfidf = np.dot(tfidf_mat, tfidf_mat.T) # 문장 간의 tf-idf

In [0]:
counter = CountVectorizer()
counter_mat = normalize(counter.fit_transform(corpus).toarray().astype(float), axis=0)
vocab = counter.vocabulary_
graph_count = np.dot(counter_mat,counter_mat.T) # 문장 간의 counter 유사도
words = {vocab[word] : word for word in vocab}

In [15]:
graph_count.shape

(1040, 1040)

In [0]:
graph = graph_count
# graph = graph_tfidf
matrix_size = graph.shape[0]

In [0]:
d=0.85
# Out(column) - In(row)
for i in range(matrix_size):
  graph[i,i] = 0 # weight 계산을 위해
  link_sum = np.sum(graph[:,i]) # i번째 sentence의 총 weight
  if link_sum != 0 :
    graph[:,i] /= link_sum # i-j weight
    graph[:,i] *= -d # 왜 -d인거지? - 수식적으로 풀면 나온다
    graph[i,i] = 1 # i-i similarity는 1이니까

In [0]:
A = normalize(x, axis=0, norm='l1')
R = np.ones(A.shape[0]).reshape(-1,1)
bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

# iteration
for _ in range(max_iter):
    R = df * (A * R) + bias

return R

In [0]:
bias = (1-d)*np.ones((matrix_size,1))

In [0]:
ranks = np.linalg.solve(graph,bias) 